In [1]:
import numpy as np

In [2]:
#Reading data
datafile = 'Part_A_Dataset/house-votes-84.data.txt'
original = np.genfromtxt(datafile,dtype='str',delimiter=',')
cols = original.copy()

for row in cols :
    yesCount = row[row=='y'].size
    noCount = row[row=='n'].size
    if(yesCount>=noCount):
        row[row=='?'] = 'y'
    else:
        row[row=='?'] = 'n'

#new, X, y = splitRandom(cols,25)   #to get parameters

In [3]:
def splitRandomWithoutReplace(cols, percent):
    rng = np.random.default_rng()
    choise = np.random.choice(cols.shape[0], int(cols.shape[0] * (percent / 100)),replace=False)
    new = cols[choise]
    rem = np.delete(cols, choise, axis=0)
    return new, rem

In [4]:
def splitRandom(cols, percent):
    rng = np.random.default_rng()
    choise = np.random.choice(cols.shape[0], int(cols.shape[0] * (percent / 100)),replace=False)
    new = cols[choise]
    rem = np.delete(cols, choise, axis=0)
    # X-->inputs Y-->outputs
    X = np.array(new[:,1:])
    y = np.array(new[:,0])
    #print(new.shape,rem.shape,cols.shape)
    return new,X,y,rem

In [5]:
class Node:
    def __init__(self):
        self.col = None
        self.remainingCols = None
        self.bestIndex = None
        self.IG = None
        self.data = None
        self.EntropyYes = None
        self.EntropyNo = None
        self.rightY = None
        self.mid = None
        self.leftN = None
        self.Class = None

    def printTree(self):
        if self.leftN:
            self.leftN.printTree()
        self.printNode()
        if self.rightY:
            self.rightY.printTree()

    def printNode(self):
        print(self.col, self.remainingCols, self.bestIndex, self.IG, self.EntropyYes, self.EntropyNo, self.rightY, self.mid, self.leftN, self.Class)
        if type(self.data) is np.ndarray:
            print(self.data.shape)
        else :
            print(None)

def treeSize(node):
    if node is None:
        return 0
    return 1 + treeSize(node.leftN) + treeSize(node.rightY)

In [6]:
def GetBestIGNode(data, colNames, mid=0):
    y = data.T[0]
    divY = y[y=='republican'].size/y.size
    ETarget = -1*( (divY)*np.log2(divY) + (1-divY)*np.log2(1-divY) )
    IGs = []
    n = data.shape[1]
    for i in range(1,n):

        #Entropy of y
        mask = np.logical_and(data.T[i]=='y', data.T[0]=='republican')
        if(data.T[i][data.T[i]=='y'].size == 0):
            div1 = 0
        else:
            div1 = data.T[i][mask].size/data.T[i][data.T[i]=='y'].size
        div2 = 1-div1
        EntropyYes = -1*((div1)*np.log2(div1) + (div2)*np.log2(div2))
        if (div1 == 0 or div1 == 1):
            EntropyYes = 0

        #Entropy of n
        mask = np.logical_and(data.T[i]=='n', data.T[0]=='republican')
        if (data.T[i][data.T[i]=='n'].size == 0):
            div3 = 0
        else:
            div3 = data.T[i][mask].size/data.T[i][data.T[i]=='n'].size
        div4 = 1-div3
        EntropyNo = -1*((div3)*np.log2(div3) + (div4)*np.log2(div4))
        if (div3 == 0 or div3 == 1):
            EntropyNo = 0
        
        #Entropy of ?
        mask = np.logical_and(data.T[i]=='?', data.T[0]=='republican')
        if (data.T[i][data.T[i]=='?'].size == 0):
            div5 = 0
        else:
            div5 = data.T[i][mask].size/data.T[i][data.T[i]=='?'].size
        div6 = 1-div5
        EntropyQ = -1*((div3)*np.log2(div3) + (div4)*np.log2(div4))
        if (div5 == 0 or div5 == 1):
            EntropyQ = 0

        #Calculate Information Gain
        IGs.append(ETarget- ( (data.T[i][data.T[i]=='y'].size/data.T[i].size)*EntropyYes + (data.T[i][data.T[i]=='n'].size/data.T[i].size)*EntropyNo + (data.T[i][data.T[i]=='?'].size/data.T[i].size)*EntropyQ ) )

    bestIndex = IGs.index(max(IGs))
    best = Node()
    best.col = colNames[bestIndex]
    best.remainingCols = colNames[:bestIndex]+colNames[bestIndex+1:]
    best.bestIndex = bestIndex + 1
    best.IG = IGs[bestIndex]
    best.data = data
    best.EntropyYes = EntropyYes
    best.EntropyNo = EntropyNo
    best.EntropyQ = EntropyNo
    if(EntropyYes == 0):
        leaf = Node()
        leaf.Class = 'republican' if div1>div2 else 'democrat'
        best.rightY = leaf
    if(EntropyNo == 0):
        leaf = Node()
        leaf.Class = 'republican' if div3>div4 else 'democrat'
        best.leftN  = leaf
    if(mid == 1 and EntropyQ == 0):
        leaf = Node()
        leaf.Class = 'republican' if div5>div6 else 'democrat'
        best.mid = leaf
    return best

In [7]:
def buildTree(rootNode, mid=0):
    #baseCase
    if(rootNode.data.shape[1] == 2):

        #right node yes
        mask = np.logical_and(data.T[1]=='y', data.T[0]=='republican')
        if(data.T[1][data.T[1]=='y'].size == 0):
            div1 = 0
        else:
            div1 = data.T[1][mask].size/data.T[1][data.T[1]=='y'].size
        leaf = Node()
        leaf.Class = 'republican' if div1>0.5 else 'democrat'
        rootNode.rightY = leaf

        #left node no
        mask = np.logical_and(data.T[1]=='n', data.T[0]=='republican')
        if (data.T[1][data.T[1]=='n'].size == 0):
            div3 = 0
        else:
            div3 = data.T[1][mask].size/data.T[1][data.T[1]=='n'].size
        leaf2 = Node()
        leaf2.Class = 'republican' if div3>0.5 else 'democrat'
        rootNode.leftN = leaf2

        #mid node ?
        if(mid == 1):
            mask = np.logical_and(data.T[1]=='?', data.T[0]=='republican')
            if (data.T[1][data.T[1]=='?'].size == 0):
                div5 = 0
            else:
                div5 = data.T[1][mask].size/data.T[1][data.T[1]=='?'].size
            leaf3 = Node()
            leaf3.Class = 'republican' if div5>0.5 else 'democrat'
            rootNode.mid = leaf3
    else:
        if(rootNode.rightY == None):
            newData = np.copy(rootNode.data)
            newData = newData[newData[:,rootNode.bestIndex] == 'y']
            newData = np.concatenate((newData[:,:rootNode.bestIndex], newData[:,rootNode.bestIndex+1:]), axis = 1)
            if newData.shape[0] == 0 :
                rootNode.Class == rootNode.data[0][0]
                return rootNode
            else:
                rootNode.rightY = buildTree(GetBestIGNode(newData,rootNode.remainingCols), mid)
        if(rootNode.leftN == None):
            newData = np.copy(rootNode.data)
            newData = newData[newData[:,rootNode.bestIndex] == 'n']
            newData = np.concatenate((newData[:,:rootNode.bestIndex], newData[:,rootNode.bestIndex+1:]), axis = 1)
            if newData.shape[0] == 0 :
                rootNode.Class == rootNode.data[0][0]
                return rootNode
            rootNode.leftN = buildTree(GetBestIGNode(newData,rootNode.remainingCols), mid)
        if(mid ==1 and rootNode.mid == None):
            newData = np.copy(rootNode.data)
            newData = newData[newData[:,rootNode.bestIndex] == '?']
            newData = np.concatenate((newData[:,:rootNode.bestIndex], newData[:,rootNode.bestIndex+1:]), axis = 1)
            if newData.shape[0] == 0 :
                rootNode.Class == rootNode.data[0][0]
                return rootNode
            rootNode.mid =buildTree(GetBestIGNode(newData,rootNode.remainingCols),1)
    return rootNode

In [8]:
def predict(rootNode, row):
    if rootNode.Class != None:
        return rootNode.Class
    else:
        if row[rootNode.bestIndex] == 'y':
            return predict(rootNode.rightY, np.concatenate((row[:rootNode.bestIndex],row[rootNode.bestIndex+1:])))
        elif row[rootNode.bestIndex] == 'n':
            return predict(rootNode.leftN, np.concatenate((row[:rootNode.bestIndex],row[rootNode.bestIndex+1:])))
        elif row[rootNode.bestIndex] == '?':
            return predict(rootNode.mid, np.concatenate((row[:rootNode.bestIndex],row[rootNode.bestIndex+1:])))

In [9]:
def predictTestData(rootNode,testData):
    right = 0
    index = 0
    for row in testData:
        right = right + (1 if predict(rootNode,row) == testData[index][0] else 0)
        index += 1
    return ((right/testData.shape[0])*100)

In [10]:
## Part_A Point_1 
## 25%rondomly and rerun 5 times and Report the sizes and accuracies of these trees in each experiment
for i in range(5):
    origin, rem = splitRandomWithoutReplace(cols,25)
    colNames = ['i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13','i14','i15','i16']
    rootNode = buildTree(GetBestIGNode(origin,colNames,1), 1)
    print('Expermint', i+1, ':')
    print('accuracy :', predictTestData(rootNode, rem))
    print('tree size :', treeSize(rootNode), '\n')

Expermint 1 :
accuracy : 94.18960244648318
tree size : 27 

Expermint 2 :
accuracy : 90.82568807339449
tree size : 19 

Expermint 3 :
accuracy : 93.27217125382263
tree size : 35 

Expermint 4 :
accuracy : 91.74311926605505
tree size : 19 

Expermint 5 :
accuracy : 93.88379204892966
tree size : 27 



In [11]:
## Part_A Point_2 
## training set sizes (30-70%) report the mean, maximum and minimum accuracies at eachtraining set size.
## Also measure the mean, max and min tree size
sizes = [30, 40, 50, 60, 70]
accuracies = []
treeSizes = []
for size in sizes:
    data, X, y, rem = splitRandom(cols,size)
    colNames = ['i1','i2','i3','i4','i5','i6','i7','i8','i9','i10','i11','i12','i13','i14','i15','i16']
    rootNode = buildTree(GetBestIGNode(data,colNames))
    accuracies.append(predictTestData(rootNode, rem))
    treeSizes.append(treeSize(rootNode))
print('Accuracies :', accuracies)
print('Tree sizes :', treeSizes)
print('mean accuracies :', np.mean(np.array(accuracies)) )
print('maximum accuracy :', np.max(np.array(accuracies)) )
print('minimum accuracies :', np.min(np.array(accuracies)) )
print('mean sizes :', np.mean(np.array(treeSizes)) )
print('maximum sizes :', np.max(np.array(treeSizes)) )
print('minimum sizes :', np.min(np.array(treeSizes)) )

Accuracies : [95.08196721311475, 96.93486590038314, 90.82568807339449, 87.93103448275862, 93.12977099236642]
Tree sizes : [19, 49, 39, 45, 68]
mean accuracies : 92.78066533240349
maximum accuracy : 96.93486590038314
minimum accuracies : 87.93103448275862
mean sizes : 44.0
maximum sizes : 68
minimum sizes : 19
